In [45]:
# import all libraries and modules
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import f1_score, accuracy_score
import requests
from urllib.parse import urlparse, parse_qs
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression


In [46]:
# with open('malicious_url.csv','r') as raw_data:
#     rows = raw_data.readlines()
#     for row in rows:
#           #url,https,www,numdots,subdomainlvl,pathlevels,urllength,numdashinhost,numat,numtide,numunderscore,numpercent,numquery,numand,numhash,numnumbric,isip,isdomaininsub,isdomaininpath,querylength,numdobleslash,isredirecting,target
#           row_data=row.split(',')
#           url = row_data[0]
#           if not url.startswith(("http://", "https://")):
#             url = "http://" + url
#           https=url.count("https://")
#           www=url.count("www.")
#           numdots=url.count(".")

#           parsed_url_subdomain = urlparse(url)
#           hostname = parsed_url_subdomain.hostname
#           subdomains = hostname.split(".")
#           subdomainlvl = len(subdomains) - 2 


#           try:
#             if (url.find("://") != -1):
#                 pathlevel = url.split('://')
#                 if (url.find("www.") != -1):
#                     pathlevel = pathlevel[1].split('www.')
#                 else:
#                     pathlevel = pathlevel[1].split('/')
#             elif (url.find("www.") != -1):
#                 pathlevel = url.split('www.')
#                 pathlevel = pathlevel[1].split('/')
#             else:
#                 pathlevel = url.split('/')

#             pathlevels = len(pathlevel) - 1
#           except Exception as e:
#             pathlevels=99


#           urllength=len(url)
#           numdash=url.count("-")
#           numat=url.count("@")
#           numtide=url.count("~")
#           numunderscore=url.count("_")
#           numpercent=url.count("%")

#           query_url = urlparse(url)
#           query_string = query_url.query
#           query_parameters = parse_qs(query_string)
#           numquery = len(query_parameters)


#           numand=url.count("&")
#           numhash=url.count("#")

#           numnumbric=0
#           for char in url:
#                if char.isdigit():
#                     numnumbric+=1
                    
#           isdomaininpath=0
#           try:
#             if ( url.find("://") != -1) or (url.find("www.") != -1)>0:
#               if(url.find("://")!= -1):
#                   urltopdomain=url.split("://")
#               if(url.find("www.")!= -1):
#                   urltopdomain=urltopdomain[1].split("www.")
#               urltopdomain=urltopdomain[1].split(".")
              
#             else:
#               urltopdomain=url.split(".")
              
#             topleveldomain=urltopdomain[0]
#             if (url.count(topleveldomain)>1):
#               isdomaininpath=1
#           except:
#              isdomaininpath=1

#           new_url=url.find("?")
#           querylength=len(url[new_url:])-1

#           numdobleslash=url.count("--")


#           target= row_data[1]
#           data_string = f"{url},{https},{www},{numdots},{subdomainlvl},{pathlevels},{urllength},{numdash},{numat},{numtide},{numunderscore},{numpercent},{numquery},{numand},{numhash},{numnumbric},{isdomaininpath},{querylength},{numdobleslash},{target}"
#           with open('data.csv', 'a', newline='') as write_raw:
#               write_raw.write(data_string)

In [47]:
data=pd.DataFrame(pd.read_csv('data.csv',nrows=4000))
data.head()
print(data['target'].unique())


['type' 'phishing' 'benign' 'defacement' 'malware']


In [48]:
#data split
X=data.iloc[:,1:19]
y=data['target']
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


In [49]:
#preprocessing

minmax=MinMaxScaler()
x_train=minmax.fit_transform(x_train)
x_test=minmax.transform(x_test)

# smote=SMOTE(random_state=42)
# x_train_resampled, y_train_resampled = smote.fit_resample(x_train, y_train)


In [50]:
rf = RandomForestClassifier(class_weight='balanced',max_depth=None,min_samples_leaf=1,min_samples_split=2,n_estimators=10)

# param_grid = {
#     'n_estimators': [10, 50, 100],
#     'max_depth': [None, 10, 20, 30],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4],
#     'class_weight': ['balanced']
# }

# gridsearch=GridSearchCV(estimator=rf,param_grid=param_grid,cv=5,scoring='f1',n_jobs=-1)
# gridsearch.fit(x_train_resampled,y_train_resampled)

# bestrf=gridsearch.best_estimator_
# bestparam=gridsearch.best_params_
# bestscore=gridsearch.best_score_

rf.fit(x_train,y_train)
y_pred=rf.predict(x_test)


# Calculate the F1 score with different average settings
f1_micro = f1_score(y_test, y_pred, average='micro')
f1_macro = f1_score(y_test, y_pred, average='macro')
f1_weighted = f1_score(y_test, y_pred, average='weighted')

# Print the accuracy and F1 scores with different averages
# print("Accuracy on Test Set:", accuracy)
print("F1 Score (Micro) on Test Set:", f1_micro)
print("F1 Score (Macro) on Test Set:", f1_macro)
print("F1 Score (Weighted) on Test Set:", f1_weighted)


accuracy = accuracy_score(y_test, y_pred)
print("Accuracy on Test Set:", accuracy)


F1 Score (Micro) on Test Set: 0.936
F1 Score (Macro) on Test Set: 0.6836583447250402
F1 Score (Weighted) on Test Set: 0.9342525150490443
Accuracy on Test Set: 0.936


In [51]:
xgb=XGBClassifier()
logregcv=LogisticRegression()
estimators=[('random_forest',rf),('xgboost',xgb)]

stackingclf=StackingClassifier(estimators=estimators,final_estimator=logregcv)
stackingclf.fit(x_train,y_train)


StackingClassifier(estimators=[('random_forest',
                                RandomForestClassifier(class_weight='balanced',
                                                       n_estimators=10)),
                               ('xgboost',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None,
                                              device=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric=None,
                                              feature_types=None, g...
                                              interaction_constraints=None,
                                              learning_rate=None, max_bin=None,
                                              max_cat_threshold=None,
                                              max_cat_to_onehot=None,
                                              max_delta_step=None,
                                              max_depth=None, max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              multi_strategy=None,
                                              n_estimators=None, n_jobs=None,
                                              num_parallel_tree=None,
                                              random_state=None, ...))],
                   final_estimator=LogisticRegression())

In [52]:
y_pred=stackingclf.predict(x_test)

accuracy=accuracy_score(y_test,y_pred)
f1_micro=f1_score(y_test,y_pred, average='micro')
f1_macro=f1_score(y_test,y_pred, average='macro')
f1_weighted=f1_score(y_test,y_pred, average='weighted')

print("accuracy",accuracy)
print("f1 micro average", f1_micro)
print("f1 micro average", f1_macro)
print("f1 micro average", f1_weighted)



accuracy 0.954
f1 micro average 0.954
f1 micro average 0.7323773934004854
f1 micro average 0.9525396850080514
